In [15]:
import findspark
findspark.init()
import pyspark

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ANN').getOrCreate()

In [17]:
spark

In [18]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [19]:
# Load training data
data = spark.read.format("libsvm")\
    .load("data/sample_multiclass_classification_data.txt")

In [20]:
# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

In [21]:
# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [4, 5, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

In [22]:
# train the model
model = trainer.fit(train)

In [23]:
# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.9523809523809523
